In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack, vstack
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix

!pip install scikit-optimize
from skopt import forest_minimize, gp_minimize

from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 81kB 2.1MB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
df = pd.read_csv("/content/drive/My Drive/Projetos/Rec_carros/clean_data.csv")
df.set_index('id', inplace= True)
df["opcionais"] = df["opcionais"].fillna("SemOpcional")
df.head()

,modelo,y,ano,valor,cor,km,opcionais,link
id,,,,,,,,
1014060,c3 hatch attraction 1 2 12v,1,17/18,43800.0,Branca,38000.0,AirBagDuplo Alarme ArCondicionado ArDigital Câ...,https://www.shopcar.com.br/veiculos/citro-n/c3...
1003530,c3 hatch attraction 1 2 12v,1,18/19,39500.0,Marrom,24536.0,AirBagDuplo Alarme ArCondicionado ArQuente Com...,https://www.shopcar.com.br/veiculos/citro-n/c3...
1002771,c3 hatch attraction 1 5,0,15/15,29900.0,Branca,47000.0,AirBag AirBagDuplo Alarme ArCondicionado ArQue...,https://www.shopcar.com.br/veiculos/citro-n/c3...
981268,c3 hatch attraction 1 5,0,14/15,29900.0,Branca,0.0,AirBag Alarme ArCondicionado ArQuente Computad...,https://www.shopcar.com.br/veiculos/citro-n/c3...
1002011,c3 hatch attraction 1 6 16v,0,19/20,62990.0,Prata,0.0,AirBagDuplo ArCondicionado ArDigital ArQuente ...,https://www.shopcar.com.br/veiculos/citro-n/c3...


#Random Forest

In [ ]:
X= df[["valor", "km"]]
y= df['y']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.5, random_state = 1)

In [ ]:
def tune_rf(params):
  print(params)
  n_estimators = params[0]

  min_df = params[1]

  colunas = ['modelo', 'ano', 'cor', 'opcionais']
  X = df[colunas]

  colunas_train, colunas_valid = train_test_split(X, test_size=0.5, random_state = 1)

  vectorizer = TfidfVectorizer(min_df=min_df)

  modelo_bow_train = vectorizer.fit_transform(colunas_train['modelo'])
  modelo_bow_valid = vectorizer.transform(colunas_valid['modelo'])

  ano_bow_train = vectorizer.fit_transform(colunas_train['ano'])
  ano_bow_valid = vectorizer.transform(colunas_valid['ano'])

#  cor_bow_train = vectorizer.fit_transform(colunas_train['cor'])
 # cor_bow_valid = vectorizer.transform(colunas_valid['cor'])

#  opcionais_bow_train = vectorizer.fit_transform(colunas_train['opcionais'])
 # opcionais_bow_valid = vectorizer.transform(colunas_valid['opcionais'])

  X_train_wbows = hstack([X_train, 
                          modelo_bow_train, 
                          ano_bow_train, 
                          #cor_bow_train, 
                          #opcionais_bow_train
                          ])

  X_valid_wbows = hstack([X_valid, 
                          modelo_bow_valid, 
                          ano_bow_valid, 
                          #cor_bow_valid, 
                          #opcionais_bow_valid
                          ])
  
  mdl = RandomForestClassifier(n_estimators=n_estimators, random_state=0, 
                               class_weight="balanced", n_jobs=-1)
  
  mdl.fit(X_train_wbows, y_train)

  p = mdl.predict_proba(X_valid_wbows)[:, 1]

  print(roc_auc_score(y_valid, p))

  return -average_precision_score(y_valid, p)

space = [(100, 1000), #n_estimators
          (1, 5), #min_df
          ]

#res = gp_minimize(tune_rf, space, random_state=3, n_random_starts=20, n_calls=40, verbose=1)

In [ ]:
[100, 10, 5]
[100, 10, 5] 7687
[725, 16, 3] 7620

</br> S/ Opcionais e Cor
</br> Ap - 0.8256 Auc - 0.9841379310344828 

</br> n_estimators = 979
</br> max_depth = 12
</br> min_df = 3
</br> ngram_range = 1

</br> S/ Opcionais
</br> Ap - 0.7835 Auc - 0.9764137931034483 

</br> n_estimators = 368
</br> max_depth = 23
</br> min_samples_split = 4
</br> max_df = 188
</br> min_df = 4
</br> ngram_range = 5

In [3]:
  #Parametros finais escolhidos
l_ap_rf = []
for seed in range(0,1):
    print(seed)
    #Cross-Validation
    X_n= df[["valor", "km"]]
    y= df['y']

    X_train, X_valid, y_train, y_valid = train_test_split(X_n, y, test_size=0.5, random_state = seed)    

    X_c = df[['modelo', 'ano']]
    colunas_train, colunas_valid = train_test_split(X_c, test_size=0.5, random_state = seed)

    #Vetorizador
    vectorizer = TfidfVectorizer(min_df=1)

    modelo_bow_train = vectorizer.fit_transform(colunas_train['modelo'])
    modelo_bow_valid = vectorizer.transform(colunas_valid['modelo'])

    ano_bow_train = vectorizer.fit_transform(colunas_train['ano'])
    ano_bow_valid = vectorizer.transform(colunas_valid['ano'])

    X_train_wbows = hstack([X_train, modelo_bow_train, ano_bow_train])
    X_valid_wbows = hstack([X_valid, modelo_bow_valid, ano_bow_valid])

    #Modelo
    mdl = RandomForestClassifier(n_estimators=700, random_state=0, 
                                  max_depth=15,
                                  class_weight="balanced", n_jobs=-1)
      
    mdl.fit(X_train_wbows, y_train)

    p_rf = mdl.predict_proba(X_valid_wbows)[:, 1]

    ap_rf = average_precision_score(y_valid, p_rf)
    l_ap_rf.append(ap_rf)
mean_ap_rf = np.mean(l_ap_rf)
mean_ap_rf

0


0.7724759242163096

In [4]:
modelo_bow_train

<315x60 sparse matrix of type '<class 'numpy.float64'>'
	with 1138 stored elements in Compressed Sparse Row format>

In [5]:
modelo_bow_valid

<315x60 sparse matrix of type '<class 'numpy.float64'>'
	with 1125 stored elements in Compressed Sparse Row format>

# Light GBM

In [ ]:
    X_n= df[["valor", "km"]]
    y= df['y']

    X_train, X_valid, y_train, y_valid = train_test_split(X_n, y, test_size=0.5, random_state = 4)

In [ ]:
def tune_lgbm(params):
    print(params)
    lr = params[0]
    max_depth = 9
    min_child_samples = 1
    subsample = 0.936
    colsample_bytree = 0.86
    n_estimators = params[1]
    
    min_df = 1

    rds = 4

    X_n= df[["valor", "km"]]
    y= df['y']

    X_train, X_valid, y_train, y_valid = train_test_split(X_n, y, test_size=0.5, random_state = rds)    

    X_c = df[['modelo', 'ano']]
    colunas_train, colunas_valid = train_test_split(X_c, test_size=0.5, random_state = rds)

    vectorizer = TfidfVectorizer(min_df=min_df)

    modelo_bow_train = vectorizer.fit_transform(colunas_train['modelo'])
    modelo_bow_valid = vectorizer.transform(colunas_valid['modelo'])

    ano_bow_train = vectorizer.fit_transform(colunas_train['ano'])
    ano_bow_valid = vectorizer.transform(colunas_valid['ano'])

    X_train_wbows = hstack([X_train, modelo_bow_train, ano_bow_train])
    X_valid_wbows = hstack([X_valid, modelo_bow_valid, ano_bow_valid])

    mdl = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                          min_child_samples=min_child_samples, subsample=subsample,
                          colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                          class_weight="balanced", n_jobs=-1)
    mdl.fit(X_train_wbows, y_train)

    p_gbm = mdl.predict_proba(X_valid_wbows)[:, 1]

    print(roc_auc_score(y_valid, p_gbm))

    return -average_precision_score(y_valid, p_gbm)


space = [(1e-3, 1e-1, 'log-uniform'), # lr
          (100,1000)] # n_estimators

#res = forest_minimize(tune_lgbm, space, random_state=160745, n_random_starts=20, n_calls=50, verbose=1)

In [ ]:
#res.x

[0.005258685383546663, 747]

In [ ]:
#lr, max_depth, min_child_samples, subsample, colsample_bytree, n_estimators, min_df 
#[0.05863662099813754, 1, 11, 0.9579762110270874, 0.8374369012162952, 477, 1] .8104 rds 1
#[0.00597022916703730, 8, 01, 0.9631166231860229, 0.8786663643738658, 351, 1] .9412 rds 0
#[0.00240625071848423, 9, 01, 0.9930167010756514, 0.8846788017161962, 582, 1] .9016 rds 2
#[0.00152997794828107, 9, 01, 0.8881164514945037, 0.6613210042159653, 818, 1] .9165 rds 3
#[0.00496283183994295, 10, 1, 0.9075149116330176, 0.9395398809363911, 879, 2] .9600 rds 4
# Parcial
#max_depth = 9, min_child_samples = 1, subsample = .936, colsample_bytree = .86, min_df = 1

#lr, n_estimators
#[0.010503031063645672, 267] rds 0 .9511
#[0.005103727185784625, 148] rds 1 .8141
#[0.010933911218290180, 472] rds 2 .8815
#[0.001483607642828346, 961] rds 3 .9166
#[0.005258685383546663, 747] rds 4 .9621
# Final
# lr = .0066, n_estimators = 500

# rds 0 .9410
# rds 1 .7472
# rds 2 .8867
# rds 3 .8957
# rds 0 .9586

In [ ]:
# Parâmetros finais


l_ap_gbm = []
for seed in range(0,50):
    print(seed)

    #Cross-Validation
    X_n= df[["valor", "km"]]
    y= df['y']

    X_train, X_valid, y_train, y_valid = train_test_split(X_n, y, test_size=0.5, random_state = seed)    

    X_c = df[['modelo', 'ano']]
    colunas_train, colunas_valid = train_test_split(X_c, test_size=0.5, random_state = seed)

    #Vetorizador
    vectorizer = TfidfVectorizer(min_df=1)

    modelo_bow_train = vectorizer.fit_transform(colunas_train['modelo'])
    modelo_bow_valid = vectorizer.transform(colunas_valid['modelo'])

    ano_bow_train = vectorizer.fit_transform(colunas_train['ano'])
    ano_bow_valid = vectorizer.transform(colunas_valid['ano'])

    X_train_wbows = hstack([X_train, modelo_bow_train, ano_bow_train])
    X_valid_wbows = hstack([X_valid, modelo_bow_valid, ano_bow_valid])

    #Modelo
    mdl = LGBMClassifier(learning_rate=0.0066, num_leaves=18, max_depth=9, 
                          min_child_samples=1, subsample=0.936,
                          colsample_bytree=0.86, bagging_freq=1, n_estimators=500, 
                          random_state=0, class_weight="balanced", n_jobs=-1)
    mdl.fit(X_train_wbows, y_train)

    p_gbm = mdl.predict_proba(X_valid_wbows)[:, 1]

    ap_gbm = average_precision_score(y_valid, p_gbm)
    
    l_ap_gbm.append(ap_gbm)
mean_ap_gbm = np.mean(l_ap_gbm)
mean_ap_gbm

0


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:478: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


1


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:478: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


2


...

48


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:478: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


49


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:478: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.8191672264206091

# Logistic Regression

In [ ]:
X_train_wbows2 = csr_matrix(X_train_wbows.copy())
X_valid_wbows2 = csr_matrix(X_valid_wbows.copy())

#scaler = StandardScaler()
scaler = MaxAbsScaler()


#X_train_wbows2[:, :2] = scaler.fit_transform(X_train_wbows2[:, :2].todense())
#X_valid_wbows2[:, :2] = scaler.transform(X_valid_wbows2[:, :2].todense())

X_train_wbows2 = scaler.fit_transform(X_train_wbows2)
X_valid_wbows2 = scaler.transform(X_valid_wbows2)

/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
mdl = LogisticRegression(n_jobs=-1, random_state=0)
mdl.fit(X_train_wbows2, y_train)

p = mdl.predict_proba(X_valid_wbows2)[:, 1]

average_precision_score(y_valid, p), roc_auc_score(y_valid, p)

(0.6230570881710282, 0.9551659944151412)

In [ ]:
#(0.5094782832828029, 0.947724039829303) MaxAbs
#(0.4679065960921416, 0.931721194879089) Standard

In [ ]:
def tune_lr(params):
  #Parametros  
  print(params)
  C = params[0]

  min_df = params[1]

  rds = 1605

  #Cross-Validation
  X_n= df[["valor", "km"]]
  y= df['y']

  X_train, X_valid, y_train, y_valid = train_test_split(X_n, y, test_size=0.5, random_state = rds)    

  X_c = df[['modelo', 'ano']]
  colunas_train, colunas_valid = train_test_split(X_c, test_size=0.5, random_state = rds)

  #Vetorização
  vectorizer = TfidfVectorizer(min_df=min_df)

  modelo_bow_train = vectorizer.fit_transform(colunas_train['modelo'])
  modelo_bow_valid = vectorizer.transform(colunas_valid['modelo'])

  ano_bow_train = vectorizer.fit_transform(colunas_train['ano'])
  ano_bow_valid = vectorizer.transform(colunas_valid['ano'])

  X_train_wbows = hstack([X_train, modelo_bow_train, ano_bow_train])
  X_valid_wbows = hstack([X_valid, modelo_bow_valid, ano_bow_valid])

  #Scaling
  X_train_wbows2 = csr_matrix(X_train_wbows.copy())
  X_valid_wbows2 = csr_matrix(X_valid_wbows.copy())

  scaler = MaxAbsScaler()

  X_train_wbows2 = scaler.fit_transform(X_train_wbows2)
  X_valid_wbows2 = scaler.transform(X_valid_wbows2)

  #Modelo
  mdl = LogisticRegression(n_jobs=-1, random_state=0, C= C,
                            solver= "liblinear")
  mdl.fit(X_train_wbows2, y_train)

  p_lr = mdl.predict_proba(X_valid_wbows2)[:, 1]

  print(roc_auc_score(y_valid, p_lr))

  return -average_precision_score(y_valid, p_lr)


space = [(0.1, 10), # C
         (1, 5)] # min_df

#res = gp_minimize(tune_lr, space, random_state=1, n_random_starts=20, n_calls=50, verbose=1)

In [ ]:
res.x

[5.677575743200029, 1]

In [ ]:
#Parâmetros escolhidos
l_ap_lr = []
for seed in range(1,51):
    print(seed)
    #Cross-Validation
    X_n= df[["valor", "km"]]
    y= df['y']

    X_train, X_valid, y_train, y_valid = train_test_split(X_n, y, test_size=0.5, random_state = seed)    

    X_c = df[['modelo', 'ano']]
    colunas_train, colunas_valid = train_test_split(X_c, test_size=0.5, random_state = seed)

    #Vetorização
    vectorizer = TfidfVectorizer(min_df=1)

    modelo_bow_train = vectorizer.fit_transform(colunas_train['modelo'])
    modelo_bow_valid = vectorizer.transform(colunas_valid['modelo'])

    ano_bow_train = vectorizer.fit_transform(colunas_train['ano'])
    ano_bow_valid = vectorizer.transform(colunas_valid['ano'])

    X_train_wbows = hstack([X_train, modelo_bow_train, ano_bow_train])
    X_valid_wbows = hstack([X_valid, modelo_bow_valid, ano_bow_valid])

    #Scaling
    X_train_wbows2 = csr_matrix(X_train_wbows.copy())
    X_valid_wbows2 = csr_matrix(X_valid_wbows.copy())

    scaler = MaxAbsScaler()

    X_train_wbows2 = scaler.fit_transform(X_train_wbows2)
    X_valid_wbows2 = scaler.transform(X_valid_wbows2)

    #Modelo
    mdl = LogisticRegression(n_jobs=-1, random_state=0, C= 6,
                              solver= "liblinear")
    mdl.fit(X_train_wbows2, y_train)

    p_lr = mdl.predict_proba(X_valid_wbows2)[:, 1]

    ap_lr = average_precision_score(y_valid, p_lr)
    l_ap_lr.append(ap_lr)
mean_ap_lr = np.mean(l_ap_lr)
mean_ap_lr


1
2
3
4
5
6
7
8
9
10


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/

11
12
13
14
15
16
17
18
19
20


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/

21
22
23
24
25
26
27
28
29
30


...

31
32
33
34
35
36
37
38
39
40


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/

41
42
43
44
45
46
47
48
49
50


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/

0.56843784548295

# Ensemble

In [ ]:
print("Média de acerto Random Forest = {}".format(mean_ap_rf))
print("Média de acerto Light GBM     = {}".format(mean_ap_gbm))
print("Média de acerto Logistc Reg   = {}".format(mean_ap_lr))

Média de acerto Random Forest = 0.7846837929153665
Média de acerto Light GBM     = 0.8191672264206091
Média de acerto Logistc Reg   = 0.56843784548295


In [ ]:
Random Forest = 0.7954513039462718
Light GBM     = 0.8191672264206091
Logistc Reg   = 0.56843784548295

In [ ]:
print("Melhor taxa de acerto Random Forest = {}".format(max(l_ap_rf)))
print("Melhor taxa de acerto Light GBM     = {}".format(max(l_ap_gbm)))
print("Melhor taxa de acerto Logistc Reg   = {}".format(max(l_ap_lr)))

Melhor taxa de acerto Random Forest = 0.9192717986979095
Melhor taxa de acerto Light GBM     = 0.958623513884324
Melhor taxa de acerto Logistc Reg   = 0.7500275974428099


In [ ]:
Melhor taxa de acerto Random Forest = 0.9211580334234797
Melhor taxa de acerto Light GBM     = 0.958623513884324
Melhor taxa de acerto Logistc Reg   = 0.7500275974428099

In [ ]:
print("Melhor taxa de acerto Random Forest = {}".format(min(l_ap_rf)))
print("Melhor taxa de acerto Light GBM     = {}".format(min(l_ap_gbm)))
print("Melhor taxa de acerto Logistc Reg   = {}".format(min(l_ap_lr)))

Melhor taxa de acerto Random Forest = 0.6268447672719316
Melhor taxa de acerto Light GBM     = 0.5692593258999216
Melhor taxa de acerto Logistc Reg   = 0.4383774748086523


In [ ]:
Pior taxa de acerto Random Forest = 0.6099082046143373
Pior taxa de acerto Light GBM     = 0.5692593258999216
Pior taxa de acerto Logistc Reg   = 0.4383774748086523

In [ ]:
p_geral = (p_gbm + p_lr + p_rf)/3

print(average_precision_score(y_valid, p_geral), roc_auc_score(y_valid, p_geral))

0.799764890795688 0.9884423897581791


In [ ]:
pd.DataFrame({'RF': p_rf, "LR": p_lr, "GMB": p_gbm}).corr()

,RF,LR,GMB
RF,1.000000,0.792646,0.072345
LR,0.792646,1.000000,0.113594
GMB,0.072345,0.113594,1.000000


In [ ]:
p_geral = (0.9*p_gbm + 0.1*p_lr)

print(average_precision_score(y_valid, p_geral), roc_auc_score(y_valid, p_geral))

0.9377933076496815 0.9946206896551725


In [ ]:
p_geral = (0.2*p_gbm + 0.8*p_rf)

print(average_precision_score(y_valid, p_geral), roc_auc_score(y_valid, p_geral))

0.8597778170282082 0.9898648648648649


In [ ]:
p_geral = (0.8*p_rf + 0.2*p_lr)

print(average_precision_score(y_valid, p_geral), roc_auc_score(y_valid, p_geral))

0.8043965504797399 0.9847083926031294
